In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [2]:
data=pd.read_csv('data/data_fixed.csv',sep =';')

In [3]:
descriptive=False
h2o2=True
save=False

In [4]:
data.head()

,Fase,Banco,Tipo de tronadura,Tipo Material,M,Dominio Estructural,Diámetro,BxS,Tiempo entre Pozos Filas ms,Fc,Tipo Explosivo,P10,P20,P30,P40,P50,P60,P70,P80,P90,P100,Este,Norte,Cota,B,S,tiempo_1,tiempo_2
0,Stage 5,2930.0,Tronadura 1,Roca 1,L4,Noreste,10.625,6.5x6.5,11-105,498.0,J,0.12,0.55,0.87,1.46,2.60,3.94,5.12,6.50,9.37,15.08,59095.2,90292.2,2930.0,6.5,6.5,11.0,105.0
1,Stage 4,2990.0,Tronadura 1,Roca 1,L4,Noreste,10.625,6.5x6.5,11-105,424.0,J,0.24,0.51,0.83,1.46,2.52,3.62,4.88,6.34,8.46,13.54,59276.7,90607.7,2990.0,6.5,6.5,11.0,105.0
2,Stage 4,2930.0,Tronadura 1,Roca 1,L4,Noreste,10.625,6.5x6.5,11-105,424.0,J,0.39,0.79,1.26,1.85,2.72,3.62,4.80,6.38,9.25,14.61,59067.4,90335.2,2930.0,6.5,6.5,11.0,105.0
3,Stage 4,2990.0,Tronadura 1,Roca 1,L4,Noreste,10.625,6.5x8,5-87,385.0,J,0.16,0.67,1.10,1.69,2.60,3.66,4.72,6.22,8.15,16.46,59278.6,90567.5,2990.0,6.5,8.0,5.0,87.0
4,Stage 2,3575.0,Tronadura 2,Roca 1,L8,Norte,10.625,7x7,5-87,507.0,K,0.59,0.79,1.02,1.34,1.89,2.91,4.06,4.88,6.38,11.46,59238.4,91671.4,3575.0,7.0,7.0,5.0,87.0


In [5]:
if descriptive==True:
    reporte = data.profile_report(title='Pandas Profiling Report: {name}'.format(name='Exploratorio_general'),correlations={"cramers": False})
    reporte.to_file(output_file="reportes/{name}.html".format(name='Exploratorio_general'))

condiciones de borde:
    FC: 100 - 1000

In [6]:
pes=['P10','P20','P30','P40','P50','P60','P70','P80','P90','P100']

In [7]:
if descriptive==True:
    for i in range(0,10):
        wihout=pes[:i]+pes[i+1:]
        reporte = data.drop(columns=wihout).profile_report(title='Pandas Profiling Report: {name}'.format(name='Exploratorio_'+pes[i]),correlations={"cramers": False})
        reporte.to_file(output_file="reportes/{name}.html".format(name='Exploratorio_'+pes[i]))

In [8]:
 #Porcentaje de datos vacios 
def missing_data(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = round((df.isnull().sum()/df.isnull().count()*100).sort_values(ascending=False),2)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

if descriptive==True:
    missing_data(data)

In [9]:
data.drop(columns=['BxS','Tiempo entre Pozos Filas ms','Cota'],errors='ignore',inplace=True)

In [10]:
data=data[data['Tipo Explosivo']!='M']

In [11]:
target=['P10','P20','P30','P40','P50','P60','P70','P80','P90','P100']
#target=['P80']

In [12]:
from h2o.automl import H2OAutoML
import h2o

In [13]:
import re
ignore_regex = re.compile('|'.join([re.escape(word) for word in target]))
x = [word for word in data.columns if not ignore_regex.search(word)]
y=target

In [14]:
if h2o2==True:
    H2O_server = h2o.init(strict_version_check=False, port=54321,  nthreads=-1)
    df = h2o.H2OFrame(data)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_222"; OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1~deb9u1-b10); OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
  Starting server from /home/jupyter/.local/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgtuep9p1
  JVM stdout: /tmp/tmpgtuep9p1/h2o_jupyter_started_from_python.out
  JVM stderr: /tmp/tmpgtuep9p1/h2o_jupyter_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.3
H2O cluster version age:,8 days
H2O cluster name:,H2O_from_python_jupyter_hbiokl
H2O cluster total nodes:,1
H2O cluster free memory:,3.134 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
def MAPE(actual, predict):
    abs_pct_error = abs((actual - predict) / actual)
    mape = abs_pct_error.mean()[0]*100
    return mape

In [ ]:
models=[]
for i in y:
    print ("modelos para: "+i)
    aml = H2OAutoML(nfolds=10,
                          seed=1234,project_name=str(i))
    aml.train(x = x, y = i, training_frame = df)
    print(aml.leaderboard)
    models.append(aml.leaderboard)
    model_path = h2o.save_model(model=aml.leader, path="models/felipin/"+str(i), force=True)
    mape_v1 = MAPE(df[y], aml.leader.predict(df))
    print("MAPE: " + str(round(mape_v1, 4))+'%')
    print ("------------------------------------------------------------")

In [ ]:
mape_v1 = MAPE(df[y], aml.leader.predict(df))
print("MAPE: " + str(round(mape_v1, 4))+'%')